# 01 Imports

In [1]:
import os
import re
import numpy  as np
import pandas as pd
import logging
import sqlite3
import requests

from bs4             import BeautifulSoup
from datetime        import datetime
from sqlalchemy      import create_engine
from IPython.display import HTML

# 02 Helper Functions

In [2]:
def jupyter_settings():
#     %matplotlib inline
    %pylab inline
#     plt.style.use( 'bmh' )
#     plt.rcParams['figure.figsize'] = [18, 8]
#     plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:97% !important; }</style>') )
    pd.options.display.max_columns = 30
    pd.options.display.max_rows = 30
    pd.options.display.float_format = lambda x: f'{x:.2f}'
    pd.set_option( 'display.expand_frame_repr', False )
#     sns.set_theme(rc={'figure.figsize': (15, 6)})
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


/home/luizmaycon/anaconda3/envs/python_ds_ao_dev/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# 03 Data Collection

In [3]:
# parameters
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0'}
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# request to URL
page = requests.get( url, headers=headers )

# Beautiful Soup object
soup = BeautifulSoup( page.text, 'html.parser' )

# make pagination
total_itens = soup.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )
page_number = np.ceil( int( total_itens ) / 36 )
url = url + '?page-size=' + str( int( page_number * 36 ) )

# new request to URL
page = requests.get( url, headers=headers )

# new Beautiful Soup object
soup = BeautifulSoup( page.text, 'html.parser' )

# ==================== Product Data ====================
products = soup.find( 'ul', class_="products-listing small" )

# product id
product_list = products.find_all( 'article', 'hm-product-item' )
product_id = [p.get( 'data-articlecode' ) for p in product_list]

# product category
product_category = [p.get( 'data-category' ) for p in product_list]

# product name
product_list = products.find_all( 'a', 'link' )
product_name = [p.get_text() for p in product_list]

# product price
product_list = products.find_all( 'span', 'price regular' )
product_price = [p.get_text() for p in product_list]

# create dataframe
data = pd.DataFrame( [product_id, product_category, product_name, product_price] ).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

data.shape

(99, 4)

In [4]:
data.head()

,product_id,product_category,product_name,product_price
0,1024256001,men_jeans_slim,Slim Jeans,$ 24.99
1,1096385002,men_jeans_loose,Loose Jeans,$ 29.99
2,1114023001,men_jeans_loose,Loose Jeans,$ 44.99
3,1114023003,men_jeans_loose,Loose Jeans,$ 44.99
4,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99


# 04 Data Collection by Product

In [5]:
# empty dataframe
df_compositions = pd.DataFrame()

# unique columns for all products
aux = []
df_pattern = pd.DataFrame( columns=['Art. No.', 'Composition', 'Fit', 'Size'] )

for i in range( len( data ) ):
    # API request
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    page = requests.get( url, headers=headers )
    
    print( 'product {} START...'.format( i ) )
    print( url )
    
    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )

    # get product colors
    product_list = soup.find_all( 'a', class_=['filter-option miniature', 'filter-option miniature active'] )
    product_id = [p.get( 'data-articlecode' ) for p in product_list]
    product_color = [p.get( 'data-color' ) for p in product_list]
    df_color = pd.DataFrame( {'product_id':product_id, 'product_color':product_color} )
    
    # get data for each color
    for j in range( len( df_color ) ):
        # API request
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html'
        page = requests.get( url, headers=headers )
                
        # Beautiful Soup object
        soup = BeautifulSoup( page.text, 'html.parser' )
        
        # product name
        product_name = soup.find( 'section', class_='product-name-price' ).find_all( 'h1' )
        product_name = product_name[0].get_text()
        
        # product price
        product_price = soup.find_all( 'span', class_='price-value' )
        product_price = re.findall( r'\d+\.?\d+', product_price[0].get_text() )[0]
        
        # ==================== compositions ====================
        product_composition_list = soup.find('div', class_='content pdp-text pdp-content').find_all( 'div' )
        product_composition = [list( filter( None, p.get_text().splitlines() ) ) for p in product_composition_list]

        # create dataframe
        df_composition = pd.DataFrame( product_composition ).T
        df_composition.columns = df_composition.iloc[0]
                
        # delete first row
        df_composition = df_composition.iloc[1:].fillna( method='ffill' )

        # Remove 'Shell:', 'Pocket lining:', 'Lining:', 'Pocket:'
        df_composition['Composition'] = df_composition['Composition'].replace( 'Shell: ', '', regex=True )
        df_composition['Composition'] = df_composition['Composition'].replace( 'Pocket lining: ', '', regex=True )
        df_composition['Composition'] = df_composition['Composition'].replace( 'Lining: ', '', regex=True )
        df_composition['Composition'] = df_composition['Composition'].replace( 'Pocket: ', '', regex=True )
        
        # garantee the same number of columns
        df_composition = pd.concat( [df_pattern, df_composition], axis=0 )

        # rename columns
        df_composition = df_composition[['Art. No.', 'Composition', 'Fit', 'Size']]
        df_composition.columns = ['product_id', 'product_composition', 'product_fit', 'product_size']
        df_composition['product_name'] = product_name
        df_composition['product_price'] = product_price
        
        # keep new coluns if it shows up
        aux = aux + df_composition.columns.tolist()
        
        # merge data
        df_composition = pd.merge( df_composition, df_color, how='left', on='product_id' )
        
        # all products
        df_compositions = pd.concat( [df_compositions, df_composition], axis=0 )
                    
# final dataframe
df_compositions['style_id'] = df_compositions['product_id'].apply( lambda x: x[:-3] )
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:] )

# scrapy_datetime
df_compositions['scrapy_datetime'] = datetime.datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )

product 0 START...
https://www2.hm.com/en_us/productpage.1024256001.html
product 1 START...
https://www2.hm.com/en_us/productpage.1096385002.html
product 2 START...
https://www2.hm.com/en_us/productpage.1114023001.html
product 3 START...
https://www2.hm.com/en_us/productpage.1114023003.html
product 4 START...
https://www2.hm.com/en_us/productpage.0985159001.html


AttributeError: 'NoneType' object has no attribute 'find_all'

In [7]:
df_compositions.head()

,product_id,product_composition,product_fit,product_size,product_name,product_price,product_color,style_id,color_id,scrapy_datetime
0,1024256001,"Cotton 99%, Spandex 1%",Slim fit,"The model is 180cm/5'11"" and wears a size 33/32",Slim Jeans,24.99,Black,1024256,001,2023-01-05 14:08:17
1,1024256001,"Polyester 65%, Cotton 35%",Slim fit,"The model is 180cm/5'11"" and wears a size 33/32",Slim Jeans,24.99,Black,1024256,001,2023-01-05 14:08:17
0,1024256002,"Cotton 99%, Spandex 1%",Slim fit,"The model is 189cm/6'2"" and wears a size 31/32",Slim Jeans,24.99,Light denim blue,1024256,002,2023-01-05 14:08:17
1,1024256002,"Polyester 65%, Cotton 35%",Slim fit,"The model is 189cm/6'2"" and wears a size 31/32",Slim Jeans,24.99,Light denim blue,1024256,002,2023-01-05 14:08:17
0,1024256003,"Cotton 99%, Spandex 1%",Slim fit,"The model is 180cm/5'11"" and wears a size 33/32",Slim Jeans,24.99,Light denim blue,1024256,003,2023-01-05 14:08:17


# 05 Data Cleaning

In [10]:
# Read data
df_data = df_compositions.dropna( subset=['product_id'] )

# ==================== Product Attributes ====================
# product_name
df_data['product_name'] = df_data['product_name'].apply( lambda x: x.replace( ' ', '_' ).lower() )

# product_price
df_data['product_price'] = df_data['product_price'].astype( float )

# product_color
df_data['product_color'] = df_data['product_color'].str.replace( ' ', '_' ).str.lower()

# fit
df_data['product_fit'] = df_data['product_fit'].apply( lambda x: x.replace( ' ', '_' ).lower() if pd.notnull( x ) else x )

# size_number
df_data['size_number'] = df_data['product_size'].apply( lambda x: re.search( '\d{3}', x ).group(0) if pd.notnull( x ) else x )

# size model
df_data['size_model'] = df_data['product_size'].str.extract( '(\d+/\\d+)' )

# # ==================== Composition ====================
# break composition and create a new dataframe for it
df1 = df_data['product_composition'].str.split( ',', expand=True )
df_ref = pd.DataFrame( index=np.arange( len( df_data ) ), columns=['cotton', 'polyester', 'spandex'] )

# cotton
df_cotton0 = pd.DataFrame(df1.loc[df1[0].str.contains('Cotton', na=True), 0])
df_cotton0.columns = ['cotton']
df_cotton1 = pd.DataFrame(df1.loc[df1[1].str.contains('Cotton', na=True), 1])
df_cotton1.columns = ['cotton']
df_cotton = df_cotton0.combine_first(df_cotton1)
df_ref = pd.concat([df_ref, df_cotton.reset_index(drop=True)], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# polyester
df_polyester = df1.loc[df1[0].str.contains('Polyester', na=True), 0]
df_polyester.name = 'polyester'
df_ref = pd.concat([df_ref, df_polyester.reset_index(drop=True)], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# spandex
df_spandex1 = pd.DataFrame(df1.loc[df1[1].str.contains('Spandex', na=True), 1])
df_spandex1.columns = ['spandex']
df_spandex2 = pd.DataFrame(df1.loc[df1[2].str.contains('Spandex', na=True), 2])
df_spandex2.columns = ['spandex']
df_spandex = df_spandex1.combine_first(df_spandex2)
df_ref = pd.concat([df_ref, df_spandex.reset_index(drop=True)], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# add product_id to df_ref
df_aux = pd.concat( [df_data['product_id'].reset_index( drop=True ), df_ref], axis=1 )

# format composition
df_aux['cotton'] = df_aux['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['polyester'] = df_aux['polyester'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['spandex'] = df_aux['spandex'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )

# final join
df_aux = df_aux.groupby( 'product_id' ).max().reset_index().fillna( 0 )
df_data = pd.merge( df_data, df_aux, on='product_id', how='left' )

# remove columns and duplicates
df_data = df_data.drop( columns=['product_size', 'product_composition'], axis=1 )
df_data = df_data.drop_duplicates().reset_index( drop=True )

df_data.shape

(186, 13)

In [11]:
df_data.head()

,product_id,product_fit,product_name,product_price,product_color,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,spandex
0,1024256001,slim_fit,slim_jeans,24.99,black,1024256,001,2023-01-05 14:08:17,180,33/32,0.99,0.80,0.01
1,1024256002,slim_fit,slim_jeans,24.99,light_denim_blue,1024256,002,2023-01-05 14:08:17,189,31/32,0.99,0.80,0.01
2,1024256003,slim_fit,slim_jeans,24.99,light_denim_blue,1024256,003,2023-01-05 14:08:17,180,33/32,0.99,0.80,0.01
3,1024256004,slim_fit,slim_jeans,24.99,denim_blue,1024256,004,2023-01-05 14:08:17,NaN,NaN,0.99,0.80,0.01
4,1024256005,slim_fit,slim_jeans,24.99,dark_blue,1024256,005,2023-01-05 14:08:17,183,31/32,0.99,0.80,0.01


# 06 Data Insert

In [14]:
# reorganize columns
data_insert = df_data[[
    'product_id',
    'style_id',
    'color_id',
    'product_name',
    'product_color',
    'product_fit',
    'product_price',
    'size_number',
    'size_model',
    'cotton',
    'polyester',
    'spandex',
    'scrapy_datetime'
]]

In [15]:
# create table query
query_showroom_schema = """
    CREATE TABLE vitrine(
        product_id      TEXT,
        style_id        TEXT,
        color_id        TEXT,
        product_name    TEXT,
        product_color   TEXT,
        produc_fit      TEXT,
        product_price   REAL,
        size_number     TEXT,
        size_model      TEXT,
        cotton          REAL,
        polyester       REAL,
        spandex         REAL,
        scrapy_datetime TEXT
    )
"""

In [16]:
# create table
conn = sqlite3.connect( 'database_hm.sqlite' )
cursor = conn.execute( query_showroom_schema )
conn.commit()

In [24]:
# create database connection
conn = create_engine( 'sqlite:////home/luizmaycon/Documentos/repos/python_ds_ao_dev/database/database_hm.sqlite', echo=False )

# data insert
data_insert.to_sql( 'vitrine', con=conn, if_exists='append', index=False )

In [25]:
query = """
    SELECT * FROM vitrine
"""
pd.read_sql_query( query, conn )

,product_id,style_id,color_id,product_name,product_color,produt_fit,product_price,size_number,size_model,cotton,polyester,spandex,scrapy_datetime
0,1024256001,1024256,001,slim_jeans,black,slim_fit,19.99,180,33/32,0.99,0.80,0.01,2022-12-29 11:52:36
1,1024256002,1024256,002,slim_jeans,light_denim_blue,slim_fit,19.99,189,31/32,0.99,0.80,0.01,2022-12-29 11:52:36
2,1024256003,1024256,003,slim_jeans,light_denim_blue,slim_fit,19.99,180,33/32,0.99,0.80,0.01,2022-12-29 11:52:36
3,1024256004,1024256,004,slim_jeans,denim_blue,slim_fit,19.99,None,None,0.99,1.00,0.01,2022-12-29 11:52:36
4,1024256005,1024256,005,slim_jeans,dark_blue,slim_fit,19.99,183,31/32,0.99,1.00,0.01,2022-12-29 11:52:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0985197005,0985197,005,slim_jeans,dark_denim_blue,slim_fit,15.99,None,None,0.99,0.00,0.01,2023-01-04 12:40:41
888,0985197006,0985197,006,slim_jeans,light_denim_blue,slim_fit,19.99,None,None,0.99,0.00,0.01,2023-01-04 12:40:41
889,0985197007,0985197,007,slim_jeans,dark_gray,slim_fit,19.99,None,None,0.99,0.00,0.01,2023-01-04 12:40:41
890,1025726002,1025726,002,relaxed_jeans,light_denim_blue,relaxed_fit,11.99,187,31/32,0.99,0.00,0.01,2023-01-04 12:40:41


# 07 Arquivos de log

In [8]:
if not os.path.exists( 'logs' ):
    os.makedirs( 'logs' )
    
logging.basicConfig(
    filename='logs/webscraping_hm.txt',
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.DEBUG )

logger = logging.getLogger( 'webscraping_hm' )

In [9]:
logger.debug( 'test DEBUG' )
logger.info( 'test INFO' )
logger.warning( 'test WARNING' )